In [ ]:
import xarray as xr
import dask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time

# Read data

In [ ]:
main_dir = '/home/disk/eos12/wycheng/'

In [ ]:
dataset = xr.open_dataset(main_dir+'data/US/dataset/dataset_CONUS.nc')

# Set countries boundaries

In [ ]:
import regionmask
import geopandas as gpd

In [ ]:
PATH_TO_SHAPEFILE = main_dir+'data/WorldCountriesBoundaries/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp'
countries = gpd.read_file(PATH_TO_SHAPEFILE)

In [ ]:
indexes = np.arange(250).tolist()
countries_mask_poly = regionmask.Regions(name = 'COUNTRY', numbers = indexes, names = countries.CNTRY_NAME[indexes], abbrevs = countries.CNTRY_NAME[indexes], outlines = list(countries.geometry.values[i] for i in range(0,countries.shape[0])))
mask = countries_mask_poly.mask(dataset.isel(Time = 0), lat_name='lat', lon_name='lon')
mask = mask.where( (mask==232) & (mask.lat<49.35) & (mask.lat>24.74)  & (mask.lon>-124.78) & (mask.lon<-66.95) )

# Plot

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
dataset['F'].where( ~np.isnan(mask) ).mean(dim='Time').plot(cmap='RdYlGn_r',norm=colors.LogNorm(vmin=0.1, vmax=10))
plt.title('Annual mean lightning frequency (WWLLN)')
plt.xlim([-125,-65])
plt.ylim([25,50])
plt.rcParams.update({'font.size': 10})

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
(dataset['TO'].sum(dim='Time')/10).where( ~np.isnan(mask) ).plot(cmap='RdYlGn_r')
plt.title('Annual mean lightning events (WWLLN)')
plt.xlim([-125,-65])
plt.ylim([25,50])
plt.rcParams.update({'font.size': 10})

In [ ]:
pcp_thrs = 0.01

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
(dataset['TO'].where(dataset['pcp']<pcp_thrs).sum(dim='Time')/10).where( ~np.isnan(mask) ).plot(cmap='RdYlGn_r')
plt.title('Annual mean dry thunderstorm events')
plt.xlim([-125,-65])
plt.ylim([25,50])
plt.rcParams.update({'font.size': 10})

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
(dataset['TO'].where(dataset['pcp']<pcp_thrs).sum(dim='Time')/dataset['TO'].sum(dim='Time')).where( ~np.isnan(mask) ).plot(cmap='RdYlGn_r')
plt.title('Dry thunderstorm fraction')
plt.xlim([-125,-65])
plt.ylim([25,50])
plt.rcParams.update({'font.size': 10})

In [ ]:
dataset_1dd = xr.open_dataset(main_dir+'data/US/dataset/dataset_CONUS_1dd.nc')

In [ ]:
date='2019-06-06'

In [ ]:
fig = plt.figure(figsize=(8,6))
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['cape'].sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, cbar_kwargs={'orientation': 'horizontal'})
plt.title('CAPE (ERA5)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

In [ ]:
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
#dataset_raw['pcp'].where(dataset_raw['pcp']>0).sel(Time='2019-10-17').plot(ax=base,cmap='rainbow',zorder=1,vmin=0, vmax=2, cbar_kwargs={'orientation': 'horizontal'})
dataset_1dd['pcp'].sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, extend='both', cbar_kwargs={'orientation': 'horizontal'})
plt.title('Precipitation (TRMM)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

In [ ]:
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['t'].isel(lev=0).sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, extend='both', cbar_kwargs={'orientation': 'horizontal'})
plt.title('Temperature at 1000hPa (ERA5)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

In [ ]:
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['q'].isel(lev=0).sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, extend='both', cbar_kwargs={'orientation': 'horizontal'})
plt.title('Specific humidity at 1000hPa (ERA5)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

In [ ]:
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['z'].isel(lev=2).sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, extend='both', cbar_kwargs={'orientation': 'horizontal'})
plt.title('Geopotential height at 500 hPa (ERA5)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

In [ ]:
fig = plt.figure(figsize=(8,6))
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['F'].where(dataset['F']>0).sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1,vmin=0, vmax=10, extend='max', cbar_kwargs={'orientation': 'horizontal'})
plt.title('Lightning Stroke Density (WWLLN)')
plt.xlim(-135, -80)
plt.ylim(20, 50)

print( dataset['F'].sel(Time=date).sel(lat=slice(32,42),lon=slice(-120,-110)).sum(dim={'lat','lon'}).values )

In [ ]:
fig = plt.figure(figsize=(8,6))
base = countries.plot(figsize=(8,6), color=(1,1,1,0.0), edgecolor='black',zorder=2)
dataset_1dd['CP'].sel(Time=date).plot(ax=base,cmap='rainbow',zorder=1, extend='both', cbar_kwargs={'orientation': 'horizontal'})
plt.title('CP')
plt.xlim(-135, -80)
plt.ylim(20, 50)

print( dataset['CP'].sel(Time=date).sel(lat=slice(32,42),lon=slice(-120,-110)).sum(dim={'lat','lon'}).values )